# Models Reassessment Notebook


This notebook demonstrates the same workflow as `scripts/model_reassessment.py` for reassessing trained models and exporting metrics.
It:
- Loads a runs CSV describing trained models.
- Resolves model checkpoints and builds configs.
- Predicts on validation/test, logs metrics, and recalibrates.
- Saves per-model figures and a consolidated metrics CSV in `uqdd/figures/...`.

Use this for interactive exploration and documentation; for batch runs, prefer the script.

## 1. Configuration

In [ ]:
import os

from uqdd.metrics import preprocess_runs, reassess_metrics
from uqdd.utils import create_logger

# Core configuration
DATA_NAME = 'papyrus'
ACTIVITY_TYPE = 'xc50'  # or 'kx'
TYPE_N_TARGETS = 'all'
PROJECT_NAME = 'notebook-reassessment'
PROJECT_OUT_NAME = f'reassess-{PROJECT_NAME}'

# Descriptors and dims
DESCRIPTOR_PROTEIN = 'ankh-large'
DESCRIPTOR_CHEMICAL = 'ecfp2048'
PROT_INPUT_DIM = 1536
CHEM_INPUT_DIM = 2048

# Paths (relative to repo root)
repo_root = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))) if os.path.basename(
    os.getcwd()) == 'notebooks' else os.path.dirname(os.path.abspath(os.getcwd()))
data_dir = os.path.join(repo_root, 'uqdd', 'data')

DATA_SPECIFIC_PATH = f"{DATA_NAME}/{ACTIVITY_TYPE}/{TYPE_N_TARGETS}"

# Input runs CSV (place your runs file name here)
RUNS_FILE_NAME = 'runs_evidential_kx.csv'  # example; change as needed
runs_path = os.path.join(data_dir, 'runs', RUNS_FILE_NAME)

# Models and output
models_dir = os.path.join(repo_root, 'uqdd', 'models', 'saved_models', DATA_SPECIFIC_PATH)
figs_out_path = os.path.join(repo_root, 'uqdd', 'figures', DATA_SPECIFIC_PATH, PROJECT_OUT_NAME)
os.makedirs(figs_out_path, exist_ok=True)
csv_out_path = os.path.join(figs_out_path, 'metrics.csv')

# Sanity
if not os.path.exists(runs_path):
    raise FileNotFoundError(f"Runs file not found: {runs_path}")

logger = create_logger(name='reassess', file_level='info', stream_level='info')
print('Repo root:', repo_root)
print('Runs path:', runs_path)
print('Models dir:', models_dir)
print('Figures out:', figs_out_path)
print('CSV out:', csv_out_path)

## 2. Load and preprocess runs

In [ ]:
runs_df = preprocess_runs(
    runs_path,
    models_dir=models_dir,
    data_name=DATA_NAME,
    activity_type=ACTIVITY_TYPE,
    descriptor_protein=DESCRIPTOR_PROTEIN,
    descriptor_chemical=DESCRIPTOR_CHEMICAL,
    data_specific_path=DATA_SPECIFIC_PATH,
    prot_input_dim=PROT_INPUT_DIM,
    chem_input_dim=CHEM_INPUT_DIM,
)
print('Runs_df preprocessed:', runs_df.shape)
runs_df.head()

## 3. Reassess, log metrics, and recalibrate

In [ ]:
# This will:
# - Iterate models from runs_df
# - Predict on test and val
# - Compute metrics and save plots under figs_out_path/<model_name>
# - Perform isotonic recalibration and log results to csv_out_path

reassess_metrics(runs_df, figs_out_path, csv_out_path, PROJECT_OUT_NAME, logger)
print('Reassessment complete. Metrics logged to:', csv_out_path)

## 4. Notes

- To change which models are reassessed, edit `RUNS_FILE_NAME` in the configuration cell.
- Outputs are grouped per model under `figs_out_path`, and consolidated into `metrics.csv`.
- This notebook uses the same functions as the script; prefer running `python scripts/model_reassessment.py` for large batches.